## Feature Engineering

Objective: 
- Check the prediction result and find the factors that might cause prediction error  
- Create features which can resolve error factors

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
DATA_PATH = "./data"

In [3]:
df = pd.read_csv(DATA_PATH+'/data_preprocessed.csv')
token = pd.read_csv(DATA_PATH+'/data_josa_removed.csv')['text']
df['token'] = token
df.head(2)

FileNotFoundError: [Errno 2] File ./data/data_preprocessed.csv does not exist: './data/data_preprocessed.csv'

In [32]:
wrong_list = []
for i in df.index:
    if df.loc[i, 'chapter'] not in df.loc[i, 'word2vec_pred_chapter']:
        wrong_list.append(i)
        
wrong_df = df.loc[wrong_list]
wrong_df.reset_index(inplace=True)
wrong_df.head(2)

,index,chapter,qtid,text,math_field,hangul_field,tag_field,name,subtitle,qplay_question_id,question_difficulty,question_type_difficulty,grade,purpose,word2vec_pred_chapter,word2vec_pred_name,token
0,7,H1S1-03,H1S1-03-08,다항식 `3x^2+kx+7`이 모든 계수가 정수인 두 일차식의 곱으로 인수분해될 때...,"['3x^2+kx+7', 'k']","['다항식', '모든', '계수', '정수', '두', '일차식', '곱', '인수...",[],조건이 주어진 식의 인수분해,NaN,4002,4,3.0,10,open,"['HSU1-09', 'H1S1-08', 'H1S1-07']","['등비수열의 활용', '이차방정식의 근의 위치', '삼차방정식의 근의 판별']","['다항식', '`', '3', 'x', '^', '2', '+', 'kx', '+..."
1,27,H1S1-05,H1S1-05-12,"`x`에 대한 이차식 `x^2+2x+4`를 복소수의 범위에서 인수분해하였을 때, 인...","['x', 'x^2+2x+4', 'A', 'B', '(A-B)^2']","['대', '이차식', '복소수', '범위', '인수분해하였', '때', '인수',...",[],이차식의 복소수범위 인수분해,NaN,3866,3,2.0,10,open,"['H1S1-01', 'H1S1-11', 'H1S1-11']","['다항식의 덧셈과 뺄셈', '접선의 길이', '원의 방정식 (2)']","['`', 'x', '`', '대', '이차식', '`', 'x', '^', '2'..."


---
wrong_df는 제 word2vec 모델로 중단원을 예측 하였을 때 전혀 맞추지 못한 문제들의 리스트입니다.  
이 worng 데이터를 쭉 살펴보시고 공통점은 무엇인지, 왜 잘못 판단 하였는지 정성적으로 분석해보시면 좋을 것 같습니다.  
모든 데이터를 다 보실 필요는 없고 (너무 힘들겁니다 ㅠ) 쭉 보시며 중요하다 생각되는 부분 위주로 분석해주세요!    

또한 오류를 유발한 factor를 어떻게 제거할 수 있을지도 고민해주시면 좋을 것 같습니다.  
(잘못 판단한 이유로는 토큰화가 잘못되었다던가, 문제에 정보가 부족하다던가, 기호가 잘못 판단되었다던가 하는 문제가 있을 것 같습니다.)  
(참고로 word2vec 모델을 사용할 때에는 name과 token이외에 아무것도 사용하지 않았습니다. 이 두 칼럼 안에 답이 있을 것 같습니다.)
---

### Example)

In [38]:
wrong_df.loc[0, 'text']

'다항식 `3x^2+kx+7`이 모든 계수가 정수인 두 일차식의 곱으로 인수분해될 때, 상수 `k`가 가질 수 있는 값 중 최솟값을 구하시오.]'

In [34]:
wrong_df.loc[0, 'token']

"['다항식', '`', '3', 'x', '^', '2', '+', 'kx', '+', '7', '`', '모든', '계수', '정수', '두', '일차식', '곱', '인수분해될', '때', ',', '상수', '`', 'k', '`', '가질', '수', '있', '값', '중', '최솟값', '구하시오', '.', ']']"

In [36]:
wrong_df.loc[0, 'name']

'조건이 주어진 식의 인수분해'

In [37]:
wrong_df.loc[0, 'word2vec_pred_name']

"['등비수열의 활용', '이차방정식의 근의 위치', '삼차방정식의 근의 판별']"

'인수분해' 라는 단어가 문제에 있음에도 불구하고 엉뚱하게 예측한 것은,  
'인수분해'가 '인수분해될' 로 token화 되어 제대로 인식되지 않은 탓인 것 같습니다.  
'될' 까지 조사로 처리하여 제거할 필요가 있을 것 같습니다.